# New Section

In [ ]:
import yfinance as yf
import ta
import pandas as pd
import numpy as np
import websocket
import json
import threading
import requests
import matplotlib.pyplot as plt
from pycoingecko import CoinGeckoAPI
from iexfinance.stocks import Stock
from polygon import WebSocketClient
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Function to Fetch Stock Data from Yahoo Finance
def fetch_stock_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
        return process_data(data)

        # Function to Fetch Crypto Data from CoinGecko
        def fetch_crypto_data(symbol):
            cg = CoinGeckoAPI()
                response = cg.get_coin_market_chart_by_id(id=symbol, vs_currency="usd", days="365")
                    prices = response["prices"]
                        data = pd.DataFrame(prices, columns=["timestamp", "Close"])
                            data["timestamp"] = pd.to_datetime(data["timestamp"], unit="ms")
                                data.set_index("timestamp", inplace=True)
                                    return process_data(data)

                                    # Function to Compute Technical Indicators
                                    def process_data(data):
                                        data["SMA_200"] = ta.trend.sma_indicator(data["Close"], window=200)
                                            data["SMA_50"] = ta.trend.sma_indicator(data["Close"], window=50)
                                                data["BB_High"] = ta.volatility.bollinger_hband(data["Close"], window=20)
                                                    data["BB_Low"] = ta.volatility.bollinger_lband(data["Close"], window=20)
                                                        data["RSI"] = ta.momentum.rsi(data["Close"], window=14)
                                                            data["MACD"] = ta.trend.macd(data["Close"])
                                                                data["MACD_signal"] = ta.trend.macd_signal(data["Close"])
                                                                    data["ATR"] = ta.volatility.average_true_range(data["High"], data["Low"], data["Close"], window=14)
                                                                        return data.dropna()

                                                                        # Function to Fetch Fundamental Data (Stock Only)
                                                                        def fetch_fundamentals(symbol):
                                                                            stock = yf.Ticker(symbol)
                                                                                info = stock.info
                                                                                    return {
                                                                                            "Market Cap": info.get("marketCap"),
                                                                                                    "P/E Ratio": info.get("trailingPE"),
                                                                                                            "Debt-to-Equity": info.get("debtToEquity"),
                                                                                                                    "Dividend Yield": info.get("dividendYield"),
                                                                                                                            "Earnings Growth": info.get("earningsGrowth"),
                                                                                                                                }

                                                                                                                                # Function to Calculate Fibonacci Retracement Levels
                                                                                                                                def fibonacci_levels(data):
                                                                                                                                    max_price = data["Close"].max()
                                                                                                                                        min_price = data["Close"].min()
                                                                                                                                            return {f"{lvl}%": min_price + (lvl/100) * (max_price - min_price) for lvl in [0, 23.6, 38.2, 50, 61.8, 100]}

                                                                                                                                            # Function to Train ML Models (Random Forest, XGBoost, LSTM)
                                                                                                                                            def train_models(data):
                                                                                                                                                data["Target"] = data["Close"].shift(-1)
                                                                                                                                                    data.dropna(inplace=True)
                                                                                                                                                        X = data.drop(["Target", "Close"], axis=1)
                                                                                                                                                            y = data["Target"]
                                                                                                                                                                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                                                                                                                                                                    # Random Forest
                                                                                                                                                                        rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

                                                                                                                                                                            rf_model.fit(X_train, y_train)

                                                                                                                                                                                # XGBoost
                                                                                                                                                                                    xgb_model = XGBRegressor(n_estimators=100, random_state=42)
                                                                                                                                                                                        xgb_model.fit(X_train, y_train)

                                                                                                                                                                                            # LSTM
                                                                                                                                                                                                X_train_lstm = np.reshape(X_train.values, (X_train.shape[0], X_train.shape[1], 1))
                                                                                                                                                                                                    X_test_lstm = np.reshape(X_test.values, (X_test.shape[0], X_test.shape[1], 1))

                                                                                                                                                                                                        lstm_model = Sequential([
                                                                                                                                                                                                                LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
                                                                                                                                                                                                                        LSTM(50, return_sequences=False),
                                                                                                                                                                                                                                Dense(25),
                                                                                                                                                                                                                                        Dense(1)
                                                                                                                                                                                                                                            ])
                                                                                                                                                                                                                                                lstm_model.compile(optimizer="adam", loss="mean_squared_error")
                                                                                                                                                                                                                                                    lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=16)

                                                                                                                                                                                                                                                        return rf_model, xgb_model, lstm_model

                                                                                                                                                                                                                                                        # Function to Generate Trade Report
                                                                                                                                                                                                                                                        def generate_report(symbol, data, fundamentals, fib_levels):
                                                                                                                                                                                                                                                            last_row = data.iloc[-1]
                                                                                                                                                                                                                                                                report = f"""
                                                                                                                                                                                                                                                                    ====== Trade Analysis Report for {symbol} ======
                                                                                                                                                                                                                                                                        📊 **Technical Analysis**:
                                                                                                                                                                                                                                                                            - Current Price: {last_row['Close']}
                                                                                                                                                                                                                                                                                - RSI: {last_row['RSI']}
                                                                                                                                                                                                                                                                                    - MACD: {last_row['MACD']}, Signal: {last_row['MACD_signal']}
                                                                                                                                                                                                                                                                                        - Bollinger Bands: Upper {last_row['BB_High']}, Lower {last_row['BB_Low']}
                                                                                                                                                                                                                                                                                            - Moving Averages: 200 SMA {last_row['SMA_200']}, 50 SMA {last_row['SMA_50']}

                                                                                                                                                                                                                                                                                                    📈 **Fundamental Analysis**:
                                                                                                                                                                                                                                                                                                        - Market Cap: {fundamentals.get("Market Cap", "N/A")}
                                                                                                                                                                                                                                                                                                            - P/E Ratio: {fundamentals.get("P/E Ratio", "N/A")}

                                                                                                                                                                                                                                                                                                                    🎯 **Fibonacci Levels**:
                                                                                                                                                                                                                                                                                                                        {fib_levels}
                                                                                                                                                                                                                                                                                                                            =====================================
                                                                                                                                                                                                                                                                                                                                """
                                                                                                                                                                                                                                                                                                                                    print(report)
                                                                                                                                                                                                                                                                                                                                        return report

                                                                                                                                                                                                                                                                                                                                        # Function to Stream Live Crypto Prices
                                                                                                                                                                                                                                                                                                                                        def stream_crypto_prices(symbol):
                                                                                                                                                                                                                                                                                                                                            def on_message(ws, message):
                                                                                                                                                                                                                                                                                                                                                    data = json.loads(message)
                                                                                                                                                                                                                                                                                                                                                            price = data.get("p", "N/A")
                                                                                                                                                                                                                                                                                                                                                                    print(f"🔹 Live {symbol.upper()} Price: ${price}")

                                                                                                                                                                                                                                                                                                                                                                        def on_open(ws):
                                                                                                                                                                                                                                                                                                                                                                                ws.send(json.dumps({"type": "subscribe", "channels": [{"name": "ticker", "product_ids": [symbol]}]}))

                                                                                                                                                                                                                                                                                                                                                                                    ws = websocket.WebSocketApp("wss://ws-feed.pro.coinbase.com", on_message=on_message, on_open=on_open)
                                                                                                                                                                                                                                                                                                                                                                                        ws.run_forever()

                                                                                                                                                                                                                                                                                                                                                                                        # Function to Stream Live Stock Prices
                                                                                                                                                                                                                                                                                                                                                                                        def stream_stock_prices(symbol, api_key):
                                                                                                                                                                                                                                                                                                                                                                                            stock = Stock(symbol, token=api_key)
                                                                                                                                                                                                                                                                                                                                                                                                while True:
                                                                                                                                                                                                                                                                                                                                                                                                        try:
                                                                                                                                                                                                                                                                                                                                                                                                                    print(f"🔹 Live {symbol.upper()} Stock Price: ${stock.get_price()}")
                                                                                                                                                                                                                                                                                                                                                                                                                            except Exception as e:
                                                                                                                                                                                                                                                                                                                                                                                                                                        print(f"Error fetching stock price: {e}")
                                                                                                                                                                                                                                                                                                                                                                                                                                                time.sleep(5)

                                                                                                                                                                                                                                                                                                                                                                                                                                                # MAIN EXECUTION
                                                                                                                                                                                                                                                                                                                                                                                                                                                if __name__ == "__main__":
                                                                                                                                                                                                                                                                                                                                                                                                                                                    asset = "crypto"  # Change to "stock" for stock market streaming
                                                                                                                                                                                                                                                                                                                                                                                                                                                        symbol = "bitcoin" if asset == "crypto" else "AAPL"
                                                                                                                                                                                                                                                                                                                                                                                                                                                            api_key = "YOUR_IEX_OR_POLYGON_API_KEY"

                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Fetch Data
                                                                                                                                                                                                                                                                                                                                                                                                                                                                    data = fetch_crypto_data(symbol) if asset == "crypto" else fetch_stock_data(symbol, "2022-01-01", "2023-01-01")

                                                                                                                                                                                                                                                                                                                                                                                                                                                                        # Fetch Fundamentals (only for stocks)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                            fundamentals = fetch_fundamentals(symbol) if asset == "stock" else {}

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Train ML Models
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    rf_model, xgb_model, lstm_model = train_models(data)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        # Calculate Fibonacci Levels
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            fib_levels = fibonacci_levels(data)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Generate Report
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    report = generate_report(symbol, data, fundamentals, fib_levels)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        # Live Price Streaming
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            threading.Thread(target=stream_crypto_prices, args=(symbol,)).start() if asset == "crypto" else threading.Thread(target=stream_stock_prices, args=(symbol, api_key)).start()